# Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
from numpy import expand_dims
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,Activation,Dropout
import math
from sklearn.metrics import mean_squared_error
import warnings 
warnings.filterwarnings('ignore')

# Loading Dataset

In [2]:
df = pd.read_csv("C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\FYP-I\\Master\\Data Collection\\CollectedData\\Final_Data (2022).csv", index_col = False)

In [3]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [4]:
df.head(7)

,Province,Country_Region,Date,Day,Month,Year,Lat,Long_,Confirmed,Deaths,...,admis_upper_vax,admis_lower_vax,admis_mean_unvax,admis_upper_unvax,admis_lower_unvax,Temp,Wind,Barometer,Visibility,Weather
0,Balochistan,Pakistan,2020-03-10,10,3,2020,28.328492,65.898403,1.0,0.0,...,0.0,0.0,25.615120,44.61881,10.791250,46.29,15.00,31.43,0.00,Haze.
1,Islamabad,Pakistan,2020-03-10,10,3,2020,33.665087,73.121219,2.0,0.0,...,0.0,0.0,0.004133,0.00000,0.000000,65.14,9.57,34.71,21.33,Clear.
2,Punjab,Pakistan,2020-03-10,10,3,2020,30.811346,72.139132,0.0,0.0,...,0.0,0.0,40.904370,89.53674,11.384460,56.40,2.44,23.67,11.88,Clear.
3,Sindh,Pakistan,2020-03-10,10,3,2020,26.009446,68.776807,0.0,0.0,...,0.0,0.0,22.339800,81.02134,6.179772,65.14,9.57,34.71,21.33,Clear.
4,Khyber Pakhtunkhwa,Pakistan,2020-03-10,10,3,2020,34.485332,72.091690,0.0,0.0,...,0.0,0.0,17.376130,39.58196,5.953569,48.80,4.64,49.20,23.21,Fog.
5,Azad Jammu and Kashmir,Pakistan,2020-03-10,10,3,2020,34.027401,73.947253,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.000000,39.86,1.86,42.43,0.00,Clear.
6,Gilgit-Baltistan,Pakistan,2020-03-10,10,3,2020,35.792146,74.982138,2.0,0.0,...,0.0,0.0,12.495310,23.41426,3.297438,32.71,3.71,23.57,12.82,Fog.


# Defining Functions For Tensor Creation (Tabular to Image)

In [5]:
def createTensors(col):
    
    sindh = df[df['Province'] == 'Sindh'][col].values
    balochistan = df[df['Province'] == 'Balochistan'][col].values
    gilgit = df[df['Province'] == 'Gilgit-Baltistan'][col].values
    islamabad = df[df['Province'] == 'Islamabad'][col].values
    punjab = df[df['Province'] == 'Punjab'][col].values
    kpk = df[df['Province'] == 'Khyber Pakhtunkhwa'][col].values
    kashmir = df[df['Province'] == 'Azad Jammu and Kashmir'][col].values
    
    sb = np.zeros(len(sindh)*2)
    gi = np.zeros(len(sindh)*2)
    pk = np.zeros(len(sindh)*2)
    ax = np.zeros(len(sindh)*2)
    
    for i in range(len(sb)):
        if(i%2 == 0):
            sb[i] = sindh[i//2]
        elif(i%2 == 1):
            sb[i] = balochistan[(i - 1)//2]
        
    for i in range(len(gi)):
        if(i%2 == 0):
            gi[i] = gilgit[i//2]
        elif(i%2 == 1):
            gi[i] = islamabad[(i - 1)//2]
        
    for i in range(len(pk)):
        if(i%2 == 0):
            pk[i] = punjab[i//2]
        elif(i%2 == 1):
            pk[i] = kpk[(i - 1)//2]
    
    for i in range(len(ax)):
        if(i%2 == 0):
            ax[i] = kashmir[i//2]
        elif(i%2 == 1):
            ax[i] = -1
    
    AllData = np.zeros((4, len(sb)))
    AllData[0] = sb
    AllData[1] = gi
    AllData[2] = pk
    AllData[3] = ax
    
    
    tempArr = np.zeros((4, 8))
    tensorsArr = np.zeros((175, 4, 8))
    
    stIndex = 0
    enIndex = 8
    for tIndex in range(175):
        for i in range(4):
            tempArr[i] = AllData[i][stIndex:enIndex]
        stIndex += 8
        enIndex += 8
        tensorsArr[tIndex] = tempArr
    
    finalTensorsArr = np.zeros((88, 8, 8))
    stIndex = 0
    enIndex = 8
    for tIndex in range(0, 176, 2):
        tfIndex = tIndex//2
        for i in range(8):
            if(i < 4):
                finalTensorsArr[tfIndex][i] = tensorsArr[tIndex][i]
            else:
                finalTensorsArr[tfIndex][i] = tensorsArr[tIndex][i-4]

    convTensors = []
    
    for i in range(88):
        convTensors.append(tf.convert_to_tensor(finalTensorsArr[i], dtype=tf.float64))

    return convTensors

In [6]:
def createYtensor(col):
    y_temp = []
    for i in range(0, len(df['Date']), 7):
        try:
            y_temp.append(df[df['Date'] == str(datetime.datetime.strptime(df['Date'].iloc[i], '%Y-%m-%d') + datetime.timedelta(days=7))[:10]][col].values)
        except:
            continue
            
    y_temp = y_temp[:696]
    
    bData = []
    iData = []
    pData = []
    sData = []
    kData = []
    aData = []
    gData = []
    
    for i in range(len(y_temp)):
        try:
            bData.append(y_temp[i][0])
            iData.append(y_temp[i][1])
            pData.append(y_temp[i][2])
            sData.append(y_temp[i][3])
            kData.append(y_temp[i][4])
            aData.append(y_temp[i][5])
            gData.append(y_temp[i][6])
        except:
            continue
        
    bSum = np.zeros(100)
    iSum = np.zeros(100)
    pSum = np.zeros(100)
    sSum = np.zeros(100)
    kSum = np.zeros(100)
    aSum = np.zeros(100)
    gSum = np.zeros(100)

    for i in range(0, len(bData) - 7, 7): 
        try:
            for j in range(7):
                if (i > 0):
    #                 print(int(i//7))
                    bSum[int(i/7)] += bData[i+j]
                    iSum[int(i/7)] += iData[i+j]
                    pSum[int(i/7)] += pData[i+j]
                    sSum[int(i/7)] += sData[i+j]
                    kSum[int(i/7)] += kData[i+j]
                    aSum[int(i/7)] += aData[i+j]
                    gSum[int(i/7)] += gData[i+j]
                else:
    #                 print(int(i))
                    bSum[int(i)] += bData[i+j]
                    iSum[int(i)] += iData[i+j]
                    pSum[int(i)] += pData[i+j]
                    sSum[int(i)] += sData[i+j]
                    kSum[int(i)] += kData[i+j]
                    aSum[int(i)] += aData[i+j]
                    gSum[int(i)] += gData[i+j]
        except:
            continue

    ySum = np.zeros((100, 7, 1))
    
    for i in range(100):
        ySum[i][0] = bSum[i] 
        ySum[i][1] = iSum[i]  
        ySum[i][2] = pSum[i] 
        ySum[i][3] = sSum[i]  
        ySum[i][4] = kSum[i] 
        ySum[i][5] = aSum[i] 
        ySum[i][6] = gSum[i] 

    yTensors = np.zeros((100, 7, 1))
    for i in range(100):
        yTensors[i] = tf.convert_to_tensor(ySum[i], dtype=tf.float64)
    
    return yTensors[:88]
    

# Defining Function For Model Creation

In [7]:
def createModel():
    model = Sequential()

    model.add(Conv2D(8, (2, 2), input_shape=(8, 8, 6)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D((2,2), padding='same'))
    # model.add(Dropout(0.25))

    model.add(Conv2D(4, (2, 2)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D((2,2), padding='same'))
    # model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(36, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(7, activation='linear'))

    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=['mean_squared_error', 'mean_absolute_error'])
    
    return model

# Data Preparation

In [8]:
comTensors = expand_dims(createTensors("Confirmed"), axis=3)
deathTensors = expand_dims(createTensors("Deaths"), axis=3)
recTensors = expand_dims(createTensors("Recovered"), axis=3)
mobTensors = expand_dims(createTensors("mobility_mean"), axis=3)
popTensors = expand_dims(createTensors("population"), axis=3)
maskTensors = expand_dims(createTensors("mask_use_mean"), axis=3)

In [9]:
X = tf.concat([comTensors, deathTensors, recTensors, mobTensors, popTensors, maskTensors], 3)

# Predicting Cumulative Confirmed Cases for each Province in next 7 days

In [10]:
y = createYtensor("Confirmed")[:88]

In [11]:
# Giving 80% data for Training
X_train = X[:70]
y_train = y[:70]

In [12]:
# Giving 20% data for Training
X_test = X[70:]
y_test = y[70:]

In [13]:
conModel = createModel()
history = conModel.fit(X_train, y_train, batch_size = 4, epochs = 3500)

Epoch 1/3500
18/18 [==============================] - 1s 2ms/step - loss: 49950205411328.0000 - mean_squared_error: 49950205411328.0000 - mean_absolute_error: 5492307.5000
Epoch 2/3500
18/18 [==============================] - 0s 2ms/step - loss: 9198877278208.0000 - mean_squared_error: 9198877278208.0000 - mean_absolute_error: 2443854.0000
Epoch 3/3500
18/18 [==============================] - 0s 2ms/step - loss: 3079158366208.0000 - mean_squared_error: 3079158366208.0000 - mean_absolute_error: 1453966.7500
Epoch 4/3500
18/18 [==============================] - 0s 1ms/step - loss: 1629847945216.0000 - mean_squared_error: 1629847945216.0000 - mean_absolute_error: 1055896.7500
Epoch 5/3500
18/18 [==============================] - 0s 2ms/step - loss: 1093656117248.0000 - mean_squared_error: 1093656117248.0000 - mean_absolute_error: 857715.9375
Epoch 6/3500
18/18 [==============================] - 0s 2ms/step - loss: 841474506752.0000 - mean_squared_error: 841474506752.0000 - mean_absolute_e

18/18 [==============================] - 0s 2ms/step - loss: 173516947456.0000 - mean_squared_error: 173516947456.0000 - mean_absolute_error: 259965.2812
Epoch 50/3500
18/18 [==============================] - 0s 2ms/step - loss: 171937103872.0000 - mean_squared_error: 171937103872.0000 - mean_absolute_error: 257008.5156
Epoch 51/3500
18/18 [==============================] - 0s 2ms/step - loss: 173031833600.0000 - mean_squared_error: 173031833600.0000 - mean_absolute_error: 260644.1094
Epoch 52/3500
18/18 [==============================] - 0s 2ms/step - loss: 173728481280.0000 - mean_squared_error: 173728481280.0000 - mean_absolute_error: 265033.7500
Epoch 53/3500
18/18 [==============================] - 0s 2ms/step - loss: 170846601216.0000 - mean_squared_error: 170846601216.0000 - mean_absolute_error: 257456.5469
Epoch 54/3500
18/18 [==============================] - 0s 2ms/step - loss: 170453270528.0000 - mean_squared_error: 170453270528.0000 - mean_absolute_error: 256617.6250
Epoch 

18/18 [==============================] - 0s 2ms/step - loss: 151133011968.0000 - mean_squared_error: 151133011968.0000 - mean_absolute_error: 252101.7969
Epoch 98/3500
18/18 [==============================] - 0s 1ms/step - loss: 147431030784.0000 - mean_squared_error: 147431030784.0000 - mean_absolute_error: 245923.7969
Epoch 99/3500
18/18 [==============================] - 0s 1ms/step - loss: 148283408384.0000 - mean_squared_error: 148283408384.0000 - mean_absolute_error: 254532.8281
Epoch 100/3500
18/18 [==============================] - 0s 1ms/step - loss: 144364486656.0000 - mean_squared_error: 144364486656.0000 - mean_absolute_error: 249634.6562
Epoch 101/3500
18/18 [==============================] - 0s 1ms/step - loss: 143489531904.0000 - mean_squared_error: 143489531904.0000 - mean_absolute_error: 248518.0312
Epoch 102/3500
18/18 [==============================] - 0s 1ms/step - loss: 146992267264.0000 - mean_squared_error: 146992267264.0000 - mean_absolute_error: 251646.4531
Epo

18/18 [==============================] - 0s 1ms/step - loss: 66859212800.0000 - mean_squared_error: 66859212800.0000 - mean_absolute_error: 179225.6562
Epoch 146/3500
18/18 [==============================] - 0s 1ms/step - loss: 63133732864.0000 - mean_squared_error: 63133732864.0000 - mean_absolute_error: 167075.9062
Epoch 147/3500
18/18 [==============================] - 0s 1ms/step - loss: 59243012096.0000 - mean_squared_error: 59243012096.0000 - mean_absolute_error: 161868.9531
Epoch 148/3500
18/18 [==============================] - 0s 1ms/step - loss: 58603978752.0000 - mean_squared_error: 58603978752.0000 - mean_absolute_error: 162012.4219
Epoch 149/3500
18/18 [==============================] - 0s 1ms/step - loss: 56528613376.0000 - mean_squared_error: 56528613376.0000 - mean_absolute_error: 158366.8125
Epoch 150/3500
18/18 [==============================] - 0s 1ms/step - loss: 55093637120.0000 - mean_squared_error: 55093637120.0000 - mean_absolute_error: 157114.0312
Epoch 151/350

18/18 [==============================] - 0s 2ms/step - loss: 16483379200.0000 - mean_squared_error: 16483379200.0000 - mean_absolute_error: 85497.0938
Epoch 195/3500
18/18 [==============================] - 0s 2ms/step - loss: 13779053568.0000 - mean_squared_error: 13779053568.0000 - mean_absolute_error: 77558.8203
Epoch 196/3500
18/18 [==============================] - 0s 1ms/step - loss: 12872693760.0000 - mean_squared_error: 12872693760.0000 - mean_absolute_error: 75754.2109
Epoch 197/3500
18/18 [==============================] - 0s 1ms/step - loss: 15852634112.0000 - mean_squared_error: 15852634112.0000 - mean_absolute_error: 83155.0391
Epoch 198/3500
18/18 [==============================] - 0s 1ms/step - loss: 14283437056.0000 - mean_squared_error: 14283437056.0000 - mean_absolute_error: 79636.6172
Epoch 199/3500
18/18 [==============================] - 0s 1ms/step - loss: 14751650816.0000 - mean_squared_error: 14751650816.0000 - mean_absolute_error: 82213.8906
Epoch 200/3500
18/1

...

18/18 [==============================] - 0s 2ms/step - loss: 575470336.0000 - mean_squared_error: 575470336.0000 - mean_absolute_error: 14394.5176
Epoch 3268/3500
18/18 [==============================] - 0s 2ms/step - loss: 542455488.0000 - mean_squared_error: 542455488.0000 - mean_absolute_error: 13876.5410
Epoch 3269/3500
18/18 [==============================] - 0s 1ms/step - loss: 527102432.0000 - mean_squared_error: 527102432.0000 - mean_absolute_error: 13209.3916
Epoch 3270/3500
18/18 [==============================] - 0s 1ms/step - loss: 867450560.0000 - mean_squared_error: 867450560.0000 - mean_absolute_error: 18593.9492
Epoch 3271/3500
18/18 [==============================] - 0s 1ms/step - loss: 409208256.0000 - mean_squared_error: 409208256.0000 - mean_absolute_error: 12636.8486
Epoch 3272/3500
18/18 [==============================] - 0s 1ms/step - loss: 468043616.0000 - mean_squared_error: 468043616.0000 - mean_absolute_error: 13422.1748
Epoch 3273/3500
18/18 [===============

18/18 [==============================] - 0s 2ms/step - loss: 586510528.0000 - mean_squared_error: 586510528.0000 - mean_absolute_error: 14098.1240
Epoch 3318/3500
18/18 [==============================] - 0s 2ms/step - loss: 321199008.0000 - mean_squared_error: 321199008.0000 - mean_absolute_error: 10582.2354
Epoch 3319/3500
18/18 [==============================] - 0s 2ms/step - loss: 651298560.0000 - mean_squared_error: 651298560.0000 - mean_absolute_error: 14474.5244
Epoch 3320/3500
18/18 [==============================] - 0s 1ms/step - loss: 687621696.0000 - mean_squared_error: 687621696.0000 - mean_absolute_error: 15302.1182
Epoch 3321/3500
18/18 [==============================] - 0s 1ms/step - loss: 335521984.0000 - mean_squared_error: 335521984.0000 - mean_absolute_error: 11484.9463
Epoch 3322/3500
18/18 [==============================] - 0s 2ms/step - loss: 416318144.0000 - mean_squared_error: 416318144.0000 - mean_absolute_error: 11603.1650
Epoch 3323/3500
18/18 [===============

18/18 [==============================] - 0s 2ms/step - loss: 183073856.0000 - mean_squared_error: 183073856.0000 - mean_absolute_error: 8092.6001
Epoch 3368/3500
18/18 [==============================] - 0s 2ms/step - loss: 323089696.0000 - mean_squared_error: 323089696.0000 - mean_absolute_error: 10563.0752
Epoch 3369/3500
18/18 [==============================] - 0s 2ms/step - loss: 166535408.0000 - mean_squared_error: 166535408.0000 - mean_absolute_error: 8293.5645
Epoch 3370/3500
18/18 [==============================] - 0s 1ms/step - loss: 158230960.0000 - mean_squared_error: 158230960.0000 - mean_absolute_error: 7704.8145
Epoch 3371/3500
18/18 [==============================] - 0s 1ms/step - loss: 212048240.0000 - mean_squared_error: 212048240.0000 - mean_absolute_error: 8538.2139
Epoch 3372/3500
18/18 [==============================] - 0s 1ms/step - loss: 511194016.0000 - mean_squared_error: 511194016.0000 - mean_absolute_error: 13173.4863
Epoch 3373/3500
18/18 [===================

18/18 [==============================] - 0s 1ms/step - loss: 333859104.0000 - mean_squared_error: 333859104.0000 - mean_absolute_error: 11263.1572
Epoch 3418/3500
18/18 [==============================] - 0s 2ms/step - loss: 200784096.0000 - mean_squared_error: 200784096.0000 - mean_absolute_error: 8575.4131
Epoch 3419/3500
18/18 [==============================] - 0s 2ms/step - loss: 201244864.0000 - mean_squared_error: 201244864.0000 - mean_absolute_error: 8514.9775
Epoch 3420/3500
18/18 [==============================] - 0s 1ms/step - loss: 287954304.0000 - mean_squared_error: 287954304.0000 - mean_absolute_error: 9983.1514
Epoch 3421/3500
18/18 [==============================] - 0s 1ms/step - loss: 664652160.0000 - mean_squared_error: 664652160.0000 - mean_absolute_error: 14596.9326
Epoch 3422/3500
18/18 [==============================] - 0s 1ms/step - loss: 485891104.0000 - mean_squared_error: 485891104.0000 - mean_absolute_error: 12068.5186
Epoch 3423/3500
18/18 [==================

18/18 [==============================] - 0s 2ms/step - loss: 187552400.0000 - mean_squared_error: 187552400.0000 - mean_absolute_error: 8696.7676
Epoch 3468/3500
18/18 [==============================] - 0s 2ms/step - loss: 104163728.0000 - mean_squared_error: 104163728.0000 - mean_absolute_error: 6739.4653
Epoch 3469/3500
18/18 [==============================] - 0s 1ms/step - loss: 311305312.0000 - mean_squared_error: 311305312.0000 - mean_absolute_error: 10285.2949
Epoch 3470/3500
18/18 [==============================] - 0s 2ms/step - loss: 459406432.0000 - mean_squared_error: 459406432.0000 - mean_absolute_error: 13061.3213
Epoch 3471/3500
18/18 [==============================] - 0s 1ms/step - loss: 812484992.0000 - mean_squared_error: 812484992.0000 - mean_absolute_error: 17691.1953
Epoch 3472/3500
18/18 [==============================] - 0s 1ms/step - loss: 1053934720.0000 - mean_squared_error: 1053934720.0000 - mean_absolute_error: 20566.3008
Epoch 3473/3500
18/18 [===============

In [14]:
predictionTest = conModel.predict(X_test)
predictionTrain = conModel.predict(X_train)

In [15]:
# Evaluation for confirmed cases
trainScore = math.sqrt(mean_squared_error(y_train.reshape(-1, 1), predictionTrain.reshape(-1, 1)))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test.reshape(-1, 1), predictionTest.reshape(-1, 1)))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 28428.34 RMSE
Test Score: 365877.54 RMSE


# Predicting Cumulative Deaths for each Province in next 7 days

In [16]:
y = createYtensor("Deaths")[:88]

In [17]:
# Giving 80% data for Training
X_train = X[:70]
y_train = y[:70]

In [18]:
# Giving 20% data for Training
X_test = X[70:]
y_test = y[70:]

In [19]:
deathModel = createModel()
deathModel.fit(X_train, y_train, batch_size = 4, epochs = 3500)

Epoch 1/3500
18/18 [==============================] - 0s 2ms/step - loss: 3648931758080.0000 - mean_squared_error: 3648931758080.0000 - mean_absolute_error: 1435464.3750
Epoch 2/3500
18/18 [==============================] - 0s 2ms/step - loss: 106252894208.0000 - mean_squared_error: 106252894208.0000 - mean_absolute_error: 250418.5938
Epoch 3/3500
18/18 [==============================] - 0s 2ms/step - loss: 13647430656.0000 - mean_squared_error: 13647430656.0000 - mean_absolute_error: 94805.1250
Epoch 4/3500
18/18 [==============================] - 0s 2ms/step - loss: 1639800064.0000 - mean_squared_error: 1639800064.0000 - mean_absolute_error: 33554.6992
Epoch 5/3500
18/18 [==============================] - 0s 2ms/step - loss: 378526336.0000 - mean_squared_error: 378526336.0000 - mean_absolute_error: 15989.4570
Epoch 6/3500
18/18 [==============================] - 0s 2ms/step - loss: 157144208.0000 - mean_squared_error: 157144208.0000 - mean_absolute_error: 8751.5977
Epoch 7/3500
18/18

18/18 [==============================] - 0s 2ms/step - loss: 118136016.0000 - mean_squared_error: 118136016.0000 - mean_absolute_error: 7153.5293
Epoch 52/3500
18/18 [==============================] - 0s 1ms/step - loss: 114121856.0000 - mean_squared_error: 114121856.0000 - mean_absolute_error: 6848.4048
Epoch 53/3500
18/18 [==============================] - 0s 2ms/step - loss: 115460976.0000 - mean_squared_error: 115460976.0000 - mean_absolute_error: 6890.6494
Epoch 54/3500
18/18 [==============================] - 0s 2ms/step - loss: 117976584.0000 - mean_squared_error: 117976584.0000 - mean_absolute_error: 7168.3940
Epoch 55/3500
18/18 [==============================] - 0s 2ms/step - loss: 128539472.0000 - mean_squared_error: 128539472.0000 - mean_absolute_error: 7321.4424
Epoch 56/3500
18/18 [==============================] - 0s 2ms/step - loss: 119623608.0000 - mean_squared_error: 119623608.0000 - mean_absolute_error: 7129.2964
Epoch 57/3500
18/18 [==============================] -

18/18 [==============================] - 0s 1ms/step - loss: 91044792.0000 - mean_squared_error: 91044792.0000 - mean_absolute_error: 6312.6436
Epoch 103/3500
18/18 [==============================] - 0s 2ms/step - loss: 89798784.0000 - mean_squared_error: 89798784.0000 - mean_absolute_error: 5938.3423
Epoch 104/3500
18/18 [==============================] - 0s 1ms/step - loss: 78156248.0000 - mean_squared_error: 78156248.0000 - mean_absolute_error: 5660.7578
Epoch 105/3500
18/18 [==============================] - 0s 1ms/step - loss: 83617920.0000 - mean_squared_error: 83617920.0000 - mean_absolute_error: 5458.4219
Epoch 106/3500
18/18 [==============================] - 0s 1ms/step - loss: 78864576.0000 - mean_squared_error: 78864576.0000 - mean_absolute_error: 5504.5029
Epoch 107/3500
18/18 [==============================] - 0s 2ms/step - loss: 79077192.0000 - mean_squared_error: 79077192.0000 - mean_absolute_error: 5484.8989
Epoch 108/3500
18/18 [==============================] - 0s 2m

18/18 [==============================] - 0s 2ms/step - loss: 53120604.0000 - mean_squared_error: 53120604.0000 - mean_absolute_error: 4751.8086
Epoch 154/3500
18/18 [==============================] - 0s 2ms/step - loss: 51379628.0000 - mean_squared_error: 51379628.0000 - mean_absolute_error: 4797.0679
Epoch 155/3500
18/18 [==============================] - 0s 2ms/step - loss: 49921324.0000 - mean_squared_error: 49921324.0000 - mean_absolute_error: 4678.4282
Epoch 156/3500
18/18 [==============================] - 0s 2ms/step - loss: 52953288.0000 - mean_squared_error: 52953288.0000 - mean_absolute_error: 4828.8325
Epoch 157/3500
18/18 [==============================] - 0s 2ms/step - loss: 81566168.0000 - mean_squared_error: 81566168.0000 - mean_absolute_error: 5758.0576
Epoch 158/3500
18/18 [==============================] - 0s 2ms/step - loss: 52009280.0000 - mean_squared_error: 52009280.0000 - mean_absolute_error: 4879.1821
Epoch 159/3500
18/18 [==============================] - 0s 2m

18/18 [==============================] - 0s 1ms/step - loss: 43150376.0000 - mean_squared_error: 43150376.0000 - mean_absolute_error: 4747.6763
Epoch 205/3500
18/18 [==============================] - 0s 2ms/step - loss: 38295120.0000 - mean_squared_error: 38295120.0000 - mean_absolute_error: 4486.7866
Epoch 206/3500
18/18 [==============================] - 0s 1ms/step - loss: 45956224.0000 - mean_squared_error: 45956224.0000 - mean_absolute_error: 4887.7222
Epoch 207/3500
18/18 [==============================] - 0s 2ms/step - loss: 55698648.0000 - mean_squared_error: 55698648.0000 - mean_absolute_error: 5487.6431
Epoch 208/3500
18/18 [==============================] - 0s 1ms/step - loss: 45483452.0000 - mean_squared_error: 45483452.0000 - mean_absolute_error: 5012.1875
Epoch 209/3500
18/18 [==============================] - 0s 1ms/step - loss: 59400728.0000 - mean_squared_error: 59400728.0000 - mean_absolute_error: 5555.4648
Epoch 210/3500
18/18 [==============================] - 0s 2m

...

18/18 [==============================] - 0s 2ms/step - loss: 207570992.0000 - mean_squared_error: 207570992.0000 - mean_absolute_error: 7195.7070
Epoch 3276/3500
18/18 [==============================] - 0s 2ms/step - loss: 207467472.0000 - mean_squared_error: 207467472.0000 - mean_absolute_error: 7193.4443
Epoch 3277/3500
18/18 [==============================] - 0s 2ms/step - loss: 207368432.0000 - mean_squared_error: 207368432.0000 - mean_absolute_error: 7191.2246
Epoch 3278/3500
18/18 [==============================] - 0s 2ms/step - loss: 207275696.0000 - mean_squared_error: 207275696.0000 - mean_absolute_error: 7189.2231
Epoch 3279/3500
18/18 [==============================] - 0s 2ms/step - loss: 207181536.0000 - mean_squared_error: 207181536.0000 - mean_absolute_error: 7187.0430
Epoch 3280/3500
18/18 [==============================] - 0s 2ms/step - loss: 207081584.0000 - mean_squared_error: 207081584.0000 - mean_absolute_error: 7184.9868
Epoch 3281/3500
18/18 [=====================

18/18 [==============================] - 0s 2ms/step - loss: 202768544.0000 - mean_squared_error: 202768544.0000 - mean_absolute_error: 7091.0010
Epoch 3326/3500
18/18 [==============================] - 0s 2ms/step - loss: 202676816.0000 - mean_squared_error: 202676816.0000 - mean_absolute_error: 7088.9761
Epoch 3327/3500
18/18 [==============================] - 0s 2ms/step - loss: 202579936.0000 - mean_squared_error: 202579936.0000 - mean_absolute_error: 7086.9233
Epoch 3328/3500
18/18 [==============================] - 0s 2ms/step - loss: 202484480.0000 - mean_squared_error: 202484480.0000 - mean_absolute_error: 7084.7671
Epoch 3329/3500
18/18 [==============================] - 0s 2ms/step - loss: 202388416.0000 - mean_squared_error: 202388416.0000 - mean_absolute_error: 7082.5645
Epoch 3330/3500
18/18 [==============================] - 0s 2ms/step - loss: 202295456.0000 - mean_squared_error: 202295456.0000 - mean_absolute_error: 7080.4912
Epoch 3331/3500
18/18 [=====================

18/18 [==============================] - 0s 2ms/step - loss: 198052480.0000 - mean_squared_error: 198052480.0000 - mean_absolute_error: 6985.6504
Epoch 3376/3500
18/18 [==============================] - 0s 2ms/step - loss: 197956896.0000 - mean_squared_error: 197956896.0000 - mean_absolute_error: 6983.5542
Epoch 3377/3500
18/18 [==============================] - 0s 2ms/step - loss: 197864064.0000 - mean_squared_error: 197864064.0000 - mean_absolute_error: 6981.4526
Epoch 3378/3500
18/18 [==============================] - 0s 2ms/step - loss: 197770672.0000 - mean_squared_error: 197770672.0000 - mean_absolute_error: 6979.3389
Epoch 3379/3500
18/18 [==============================] - 0s 2ms/step - loss: 197675888.0000 - mean_squared_error: 197675888.0000 - mean_absolute_error: 6977.3486
Epoch 3380/3500
18/18 [==============================] - 0s 2ms/step - loss: 197581968.0000 - mean_squared_error: 197581968.0000 - mean_absolute_error: 6975.2622
Epoch 3381/3500
18/18 [=====================

18/18 [==============================] - 0s 2ms/step - loss: 193410576.0000 - mean_squared_error: 193410576.0000 - mean_absolute_error: 6880.2935
Epoch 3426/3500
18/18 [==============================] - 0s 2ms/step - loss: 193318176.0000 - mean_squared_error: 193318176.0000 - mean_absolute_error: 6878.1455
Epoch 3427/3500
18/18 [==============================] - 0s 2ms/step - loss: 193227856.0000 - mean_squared_error: 193227856.0000 - mean_absolute_error: 6876.2266
Epoch 3428/3500
18/18 [==============================] - 0s 2ms/step - loss: 193135472.0000 - mean_squared_error: 193135472.0000 - mean_absolute_error: 6873.8281
Epoch 3429/3500
18/18 [==============================] - 0s 2ms/step - loss: 193043728.0000 - mean_squared_error: 193043728.0000 - mean_absolute_error: 6871.7690
Epoch 3430/3500
18/18 [==============================] - 0s 2ms/step - loss: 192953648.0000 - mean_squared_error: 192953648.0000 - mean_absolute_error: 6869.6543
Epoch 3431/3500
18/18 [=====================

18/18 [==============================] - 0s 2ms/step - loss: 188827280.0000 - mean_squared_error: 188827280.0000 - mean_absolute_error: 6774.6548
Epoch 3476/3500
18/18 [==============================] - 0s 2ms/step - loss: 188733008.0000 - mean_squared_error: 188733008.0000 - mean_absolute_error: 6772.4170
Epoch 3477/3500
18/18 [==============================] - 0s 2ms/step - loss: 188642512.0000 - mean_squared_error: 188642512.0000 - mean_absolute_error: 6770.4370
Epoch 3478/3500
18/18 [==============================] - 0s 2ms/step - loss: 188555408.0000 - mean_squared_error: 188555408.0000 - mean_absolute_error: 6768.3633
Epoch 3479/3500
18/18 [==============================] - 0s 2ms/step - loss: 188464480.0000 - mean_squared_error: 188464480.0000 - mean_absolute_error: 6766.4292
Epoch 3480/3500
18/18 [==============================] - 0s 2ms/step - loss: 188377456.0000 - mean_squared_error: 188377456.0000 - mean_absolute_error: 6764.3306
Epoch 3481/3500
18/18 [=====================

In [20]:
predictionTest = deathModel.predict(X_test)
predictionTrain = deathModel.predict(X_train)

In [21]:
# Evaluation for deaths
trainScore = math.sqrt(mean_squared_error(y_train.reshape(-1, 1), predictionTrain.reshape(-1, 1)))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test.reshape(-1, 1), predictionTest.reshape(-1, 1)))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 13657.62 RMSE
Test Score: 133902.89 RMSE


# Predicting Cumulative Recovered Cases for each Province in next 7 days

In [22]:
y = createYtensor("Recovered")[:88]

In [23]:
# Giving 80% data for Training
X_train = X[:70]
y_train = y[:70]

In [24]:
# Giving 20% data for Training
X_test = X[70:]
y_test = y[70:]

In [25]:
recModel = createModel()
recModel.fit(X_train, y_train, batch_size = 4, epochs = 3500)

Epoch 1/3500
18/18 [==============================] - 0s 2ms/step - loss: 14492236775424.0000 - mean_squared_error: 14492236775424.0000 - mean_absolute_error: 3178343.7500
Epoch 2/3500
18/18 [==============================] - 0s 2ms/step - loss: 1439100829696.0000 - mean_squared_error: 1439100829696.0000 - mean_absolute_error: 974700.4375
Epoch 3/3500
18/18 [==============================] - 0s 2ms/step - loss: 303244673024.0000 - mean_squared_error: 303244673024.0000 - mean_absolute_error: 367573.4375
Epoch 4/3500
18/18 [==============================] - 0s 2ms/step - loss: 166218301440.0000 - mean_squared_error: 166218301440.0000 - mean_absolute_error: 274448.7812
Epoch 5/3500
18/18 [==============================] - 0s 2ms/step - loss: 157413048320.0000 - mean_squared_error: 157413048320.0000 - mean_absolute_error: 249949.3438
Epoch 6/3500
18/18 [==============================] - 0s 2ms/step - loss: 159244468224.0000 - mean_squared_error: 159244468224.0000 - mean_absolute_error: 236

18/18 [==============================] - 0s 2ms/step - loss: 128667303936.0000 - mean_squared_error: 128667303936.0000 - mean_absolute_error: 214264.9219
Epoch 50/3500
18/18 [==============================] - 0s 2ms/step - loss: 133007368192.0000 - mean_squared_error: 133007368192.0000 - mean_absolute_error: 209223.3125
Epoch 51/3500
18/18 [==============================] - 0s 2ms/step - loss: 126577696768.0000 - mean_squared_error: 126577696768.0000 - mean_absolute_error: 208754.0000
Epoch 52/3500
18/18 [==============================] - 0s 2ms/step - loss: 131941687296.0000 - mean_squared_error: 131941687296.0000 - mean_absolute_error: 217184.5469
Epoch 53/3500
18/18 [==============================] - 0s 2ms/step - loss: 133770543104.0000 - mean_squared_error: 133770543104.0000 - mean_absolute_error: 212302.1406
Epoch 54/3500
18/18 [==============================] - 0s 2ms/step - loss: 116067827712.0000 - mean_squared_error: 116067827712.0000 - mean_absolute_error: 203088.4219
Epoch 

18/18 [==============================] - 0s 2ms/step - loss: 8231888896.0000 - mean_squared_error: 8231888896.0000 - mean_absolute_error: 56648.3789
Epoch 99/3500
18/18 [==============================] - 0s 2ms/step - loss: 7793678336.0000 - mean_squared_error: 7793678336.0000 - mean_absolute_error: 54922.8125
Epoch 100/3500
18/18 [==============================] - 0s 2ms/step - loss: 7656860672.0000 - mean_squared_error: 7656860672.0000 - mean_absolute_error: 52861.5469
Epoch 101/3500
18/18 [==============================] - 0s 2ms/step - loss: 11143398400.0000 - mean_squared_error: 11143398400.0000 - mean_absolute_error: 65547.0078
Epoch 102/3500
18/18 [==============================] - 0s 2ms/step - loss: 7024025600.0000 - mean_squared_error: 7024025600.0000 - mean_absolute_error: 52695.6602
Epoch 103/3500
18/18 [==============================] - 0s 2ms/step - loss: 7201070592.0000 - mean_squared_error: 7201070592.0000 - mean_absolute_error: 52107.0273
Epoch 104/3500
18/18 [========

18/18 [==============================] - 0s 2ms/step - loss: 9509381120.0000 - mean_squared_error: 9509381120.0000 - mean_absolute_error: 61317.9492
Epoch 148/3500
18/18 [==============================] - 0s 2ms/step - loss: 9334467584.0000 - mean_squared_error: 9334467584.0000 - mean_absolute_error: 60132.9727
Epoch 149/3500
18/18 [==============================] - 0s 2ms/step - loss: 12033864704.0000 - mean_squared_error: 12033864704.0000 - mean_absolute_error: 69667.2109
Epoch 150/3500
18/18 [==============================] - 0s 2ms/step - loss: 13183304704.0000 - mean_squared_error: 13183304704.0000 - mean_absolute_error: 71113.4922
Epoch 151/3500
18/18 [==============================] - 0s 2ms/step - loss: 7484020736.0000 - mean_squared_error: 7484020736.0000 - mean_absolute_error: 55300.5078
Epoch 152/3500
18/18 [==============================] - 0s 2ms/step - loss: 6156592640.0000 - mean_squared_error: 6156592640.0000 - mean_absolute_error: 51657.4023
Epoch 153/3500
18/18 [=====

18/18 [==============================] - 0s 2ms/step - loss: 12829132800.0000 - mean_squared_error: 12829132800.0000 - mean_absolute_error: 71044.1016
Epoch 197/3500
18/18 [==============================] - 0s 2ms/step - loss: 6478670848.0000 - mean_squared_error: 6478670848.0000 - mean_absolute_error: 52266.8906
Epoch 198/3500
18/18 [==============================] - 0s 2ms/step - loss: 9106412544.0000 - mean_squared_error: 9106412544.0000 - mean_absolute_error: 60755.4141
Epoch 199/3500
18/18 [==============================] - 0s 2ms/step - loss: 7964593664.0000 - mean_squared_error: 7964593664.0000 - mean_absolute_error: 57425.8398
Epoch 200/3500
18/18 [==============================] - 0s 2ms/step - loss: 15863386112.0000 - mean_squared_error: 15863386112.0000 - mean_absolute_error: 78306.6328
Epoch 201/3500
18/18 [==============================] - 0s 2ms/step - loss: 13335508992.0000 - mean_squared_error: 13335508992.0000 - mean_absolute_error: 73505.2344
Epoch 202/3500
18/18 [===

...

18/18 [==============================] - 0s 2ms/step - loss: 1329179392.0000 - mean_squared_error: 1329179392.0000 - mean_absolute_error: 20962.4043
Epoch 3262/3500
18/18 [==============================] - 0s 2ms/step - loss: 1693149056.0000 - mean_squared_error: 1693149056.0000 - mean_absolute_error: 23394.6777
Epoch 3263/3500
18/18 [==============================] - 0s 2ms/step - loss: 1047471616.0000 - mean_squared_error: 1047471616.0000 - mean_absolute_error: 19995.5957
Epoch 3264/3500
18/18 [==============================] - 0s 2ms/step - loss: 824597760.0000 - mean_squared_error: 824597760.0000 - mean_absolute_error: 18206.1152
Epoch 3265/3500
18/18 [==============================] - 0s 2ms/step - loss: 495177792.0000 - mean_squared_error: 495177792.0000 - mean_absolute_error: 13709.6758
Epoch 3266/3500
18/18 [==============================] - 0s 2ms/step - loss: 334742880.0000 - mean_squared_error: 334742880.0000 - mean_absolute_error: 11088.5410
Epoch 3267/3500
18/18 [=========

18/18 [==============================] - 0s 2ms/step - loss: 437442048.0000 - mean_squared_error: 437442048.0000 - mean_absolute_error: 13083.9326
Epoch 3312/3500
18/18 [==============================] - 0s 2ms/step - loss: 245477792.0000 - mean_squared_error: 245477792.0000 - mean_absolute_error: 9934.6855
Epoch 3313/3500
18/18 [==============================] - 0s 2ms/step - loss: 248154528.0000 - mean_squared_error: 248154528.0000 - mean_absolute_error: 9074.6104
Epoch 3314/3500
18/18 [==============================] - 0s 2ms/step - loss: 434163904.0000 - mean_squared_error: 434163904.0000 - mean_absolute_error: 12159.0391
Epoch 3315/3500
18/18 [==============================] - 0s 2ms/step - loss: 807485312.0000 - mean_squared_error: 807485312.0000 - mean_absolute_error: 17410.5586
Epoch 3316/3500
18/18 [==============================] - 0s 2ms/step - loss: 780017792.0000 - mean_squared_error: 780017792.0000 - mean_absolute_error: 14694.5547
Epoch 3317/3500
18/18 [=================

18/18 [==============================] - 0s 2ms/step - loss: 303812160.0000 - mean_squared_error: 303812160.0000 - mean_absolute_error: 10562.2988
Epoch 3362/3500
18/18 [==============================] - 0s 2ms/step - loss: 280343232.0000 - mean_squared_error: 280343232.0000 - mean_absolute_error: 10016.9355
Epoch 3363/3500
18/18 [==============================] - 0s 2ms/step - loss: 561395136.0000 - mean_squared_error: 561395136.0000 - mean_absolute_error: 13209.6826
Epoch 3364/3500
18/18 [==============================] - 0s 2ms/step - loss: 315319552.0000 - mean_squared_error: 315319552.0000 - mean_absolute_error: 10589.2168
Epoch 3365/3500
18/18 [==============================] - 0s 2ms/step - loss: 289768576.0000 - mean_squared_error: 289768576.0000 - mean_absolute_error: 10515.5889
Epoch 3366/3500
18/18 [==============================] - 0s 2ms/step - loss: 598877184.0000 - mean_squared_error: 598877184.0000 - mean_absolute_error: 13945.4893
Epoch 3367/3500
18/18 [===============

18/18 [==============================] - 0s 2ms/step - loss: 286052224.0000 - mean_squared_error: 286052224.0000 - mean_absolute_error: 9916.5098
Epoch 3412/3500
18/18 [==============================] - 0s 2ms/step - loss: 822209280.0000 - mean_squared_error: 822209280.0000 - mean_absolute_error: 16326.7217
Epoch 3413/3500
18/18 [==============================] - 0s 2ms/step - loss: 541913728.0000 - mean_squared_error: 541913728.0000 - mean_absolute_error: 15381.1768
Epoch 3414/3500
18/18 [==============================] - 0s 2ms/step - loss: 219144416.0000 - mean_squared_error: 219144416.0000 - mean_absolute_error: 9300.3242
Epoch 3415/3500
18/18 [==============================] - 0s 2ms/step - loss: 214428544.0000 - mean_squared_error: 214428544.0000 - mean_absolute_error: 8677.7119
Epoch 3416/3500
18/18 [==============================] - 0s 2ms/step - loss: 232262128.0000 - mean_squared_error: 232262128.0000 - mean_absolute_error: 9744.1260
Epoch 3417/3500
18/18 [===================

18/18 [==============================] - 0s 2ms/step - loss: 377389952.0000 - mean_squared_error: 377389952.0000 - mean_absolute_error: 13033.0625
Epoch 3462/3500
18/18 [==============================] - 0s 4ms/step - loss: 296051648.0000 - mean_squared_error: 296051648.0000 - mean_absolute_error: 10332.2773
Epoch 3463/3500
18/18 [==============================] - 0s 2ms/step - loss: 409433568.0000 - mean_squared_error: 409433568.0000 - mean_absolute_error: 11993.9736
Epoch 3464/3500
18/18 [==============================] - 0s 3ms/step - loss: 1035334464.0000 - mean_squared_error: 1035334464.0000 - mean_absolute_error: 18568.0703
Epoch 3465/3500
18/18 [==============================] - ETA: 0s - loss: 2895443968.0000 - mean_squared_error: 2895443968.0000 - mean_absolute_error: 30387.96 - 0s 3ms/step - loss: 2840847872.0000 - mean_squared_error: 2840847872.0000 - mean_absolute_error: 30094.3320
Epoch 3466/3500
18/18 [==============================] - 0s 3ms/step - loss: 666320320.0000 -

In [26]:
predictionTest = recModel.predict(X_test)
predictionTrain = recModel.predict(X_train)

In [27]:
# Evaluation for recovered cases
trainScore = math.sqrt(mean_squared_error(y_train.reshape(-1, 1), predictionTrain.reshape(-1, 1)))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test.reshape(-1, 1), predictionTest.reshape(-1, 1)))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 23962.91 RMSE
Test Score: 258891.96 RMSE
